# Depuracion datos Inventario Forestal Nacional

El presente notebook realiza la exploración inicial y depuración de datos del Inventario Forestal Nacional.

Se requieren los siguientes tablas enformato csv:

1. `detritos`: Contiene los mediciones realizadas en detritos de madera.

2. `ampie`: Mediciones de árboles muertos en pie.
    
3. `vegetacion`: Mediciones de árboles vivos. 

In [9]:
import pandas as pd
import numpy as np
%matplotlib inline

# Asignar nombres de archivos a variables
#detritos = "../data/IFN/detritos.csv"
#ampie =  "../data/IFN/dasometricos/amp.csv"
vegetacion = "../data/IFN/dasometricos/vegetacion.csv"
generalInfo = "../data/IFN/informacion_general/informacion_general.csv"

# Leer archivos como Pandas dataframes
#det = pd.read_csv(detritos) 
#amp = pd.read_csv(ampie)
veg = pd.read_csv(vegetacion)
info = pd.read_csv(generalInfo, encoding = 'utf8')

# Detritos

In [ ]:
# Campos de mediciones de detritos

CONS = "CONS" # Indice de medicion (int64)
PLOT = "PLOT" # Indice conglomerado (int64)
TRAN = "TRAN" # Transecto de detritos (str: 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'). Hay registros sin transectos!!!
SECC = "SECC" # Seccion del transecto donde se registro la pieza (str: 'II', 'III', 'I'). Hay registros sin seccion!!!!
PIEZA = "PIEZA" # Numero consecutivo de la pieza en el transecto (float64). Deberia ser int64.
TIPO = "TIPO" # Tipo de detrito (str: 'DFM', 'DGM')
DIST = "DIST" # Distancia del detrito en cada seccion (float64 0.0-10.0)
AZIMUT = "AZIMUT" # Orientacion de la pieza respecto al transecto ?????????? (float64?, 0-360)
D1 = "D1" # Primer diametro de la pieza en cm (float64)
D2 = "D2" # Segundo diametro de la pieza en cm (float64)
INCL = "INCL" # Inclinacion de la pieza (float64). De acuerdo al manual deberia estar en el rango [-90, 90] pero en la hoja de calculo esta sesgada a valores positivos y hay valores [90, 180].
PI_cm = "PI_cm" # Penetracion del penetrometro en cm (float64). Verificar valores maximos.
PI_golpes = "PI_golpes" # Golpes ejecutados con el penetrometro (float64). Por que es un numero real????
PESO_RODAJA = "PESO_RODAJA" # Peso de la rodaja en gr (float64)
ESP1 = "ESP1" # Primer espesor de la rodaja en cm (float64)
ESP2 = "ESP2" # Segundo espesor de la rodaja en cm (float64)
ESP3 = "ESP3" # Tercer espesor de la rodaja en cm (float64)
ESP4 = "ESP4" # Cuarto espesor de la rodaja en cm (float64)
PESO_MUESTRA = "PESO_MUESTRA" # Peso fresco de la muestra en gr (float64)
VOL = "VOL" # Volumen de la muestra en ml (float64). Este campo no esta incluido en los formatos del INF!!!!
PESO_SECO = "PESO_SECO" # Peso fresco de la muestra en gr (float64)
DENS = "DENS" # Densidad de madera de la muestra en gr/ml (float64)

In [ ]:
# Convertir seccion muestreo de detritos en enteros para ahorrar espacio en db
try:
    det[SECC].replace(to_replace = ['I','II','III'], value = [1,2,3], inplace = True)
    # Si no existieran valores faltantes el reemplazo produciria una serie de int64, 
    # de lo contrario la columna resultante es float64
except:
    pass

for fi in [CONS, PLOT, SECC]:
    if det[fi].dtype != np.int64:
        print "Campo {0} tiene tipo inapropiado ({1} en vez de int64).".format(fi, det[fi].dtype)
        if len(det[fi][det[fi].isna()]) > 1:
            print "Valores nulos son considerados np.float64:"
            print det[[fi, PLOT]][det[fi].isna()].join(info[['PLOT', 'SOCIO']].set_index(PLOT), 
                                                       on=PLOT, rsuffix='_info')
        else:
            print "Valores np.float64 a revisar:"
            print det[fi][det[fi].map(lambda x: x % 1.0 != 0)].dropna()
            
        
for fi in [AZIMUT, PIEZA, DIST, D1, D2, INCL, PI_cm, PESO_RODAJA, ESP1, ESP2, ESP3, ESP4, 
           PESO_MUESTRA, VOL, PESO_SECO, DENS, PI_golpes]:
    if det[fi].dtype != np.float64:
        print "Campo {0} tiene tipo inapropiado ({1}en vez de float64).".format(fi, det[fi].dtype)

for fi in [TRAN, TIPO]:
    non_strings = det[fi].dropna()[~det[fi].dropna().apply(type).eq(str)]
    if len(non_strings):
        print "Campo {0} tiene tipo inapropiado ({1} en vez de str).".format(fi, non_strings.dtype)


In [ ]:
# Indices no debe contener duplicado
if len(det[det[CONS].duplicated()]):
    print "Tabla {0} contiene indices duplicados.".format(detritos)
        
for tr in det[TRAN].dropna().unique():
    if tr not in ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']:
        print "`{0}` no es un valor valido de transecto de detrito".format(tr)
        
if len(det[det[TRAN].isna()]):
    print "Piezas de detritos no tienen transecto:"
    print det[[PLOT, CONS, TRAN]][det[TRAN].isna()].join(info[['PLOT', 'SOCIO']].set_index(PLOT), 
                                                         on=PLOT, rsuffix='_info')
        
for tr in det[SECC].dropna().unique():
    if tr not in [1,2,3]:
        print "`{0}` no es un valor valido de transecto de detrito".format(tr)

for tr in det[TIPO].dropna().unique():
    if tr not in ['DFM', 'DGM']:
        print "`{0}` no es un valor valido de tipo de detrito".format(tr)
        
if len(det[(det[TIPO] == 'DFM') & (((det[D1] >= 20) & (det[D2].isna())) | 
                                   (det[D1] + det[D2] >= 40))][[TIPO, D1, D2]]):
    print "Tipo de detrito probablemente mal asignado"
    print det[[TIPO, D1, D2, PLOT]][(det[TIPO] == 'DFM') & (((det[D1] >= 20) & (det[D2].isna()))
            | (det[D1] + det[D2] >= 40))].join(info[['PLOT', 'SOCIO']].set_index(PLOT), 
            on=PLOT, rsuffix='_info')

if len(det[(det[TIPO] == 'DGM') & (((det[D1] < 20) & (det[D2].isna())) | (det[D1] + det[D2] < 40))
          ][[TIPO, D1, D2]]):
    print "Tipo de detrito probablemente mal asignado"
    print det[[TIPO, D1, D2, PLOT]][(det[TIPO] == 'DGM') & (((det[D1] < 20) & (det[D2].isna())) 
            | (det[D1] + det[D2] < 40))].join(info[['PLOT', 'SOCIO']].set_index(PLOT), 
            on=PLOT, rsuffix='_info')

if det[DIST].min() < 0:
    print "Rango distancia tiene valores negativos."
if det[DIST].max() > 10:
    print "Rango distancia sobrepasa el valor permitido (10)."
    print det[[DIST, PLOT]][det[DIST] > 10].join(info[['PLOT', 'SOCIO']].set_index(PLOT), on=PLOT, 
            rsuffix='_info')

# Distancias de detritos finos son depuradas de acuerdo a la guia de toma de datos del IFN. 
# Teoricamente, este subset de distancias deben encontrarse en el rango [9.0--10.0], pues
# son observaciones realizadas unicamente en el ultimo metro de cada seccion de detritos.
# De esta manera se sume que todos los valores encontrados en el rango [0.0--1.0] corresponden
# a mediciones realizadas con el punto de referencia equivocado, extremo final de la seccion
# en lugar del punto inicial.
det.loc[(det[TIPO] == 'DFM') & (det[DIST] <= 1), DIST] = det[DIST][(det[TIPO] == 'DFM') & 
    (det[DIST] < 1)].apply(lambda x: (1.0 - x) + 9)

if len(det[(det[TIPO] == 'DFM') & (det[DIST] > 1) & (det[DIST] < 9)]):
    print "Detritos finos contienen valores no permitidos de distancia:"
    print det[[TIPO, DIST, PLOT]][(det[TIPO] == 'DFM') & (det[DIST] > 1) & (det[DIST] < 9)
            ].join(info[['PLOT', 'SOCIO']].set_index(PLOT), on=PLOT, rsuffix='_info')

if det[AZIMUT].min() < 0:
    print "Rango Azimut tiene valores negativos."
if det[AZIMUT].max() > 360:
    print "Rango Azimut sobrepasa el valor permitido (360)"

# Valores de diametro 0.0 deben ser nulos
det[D2].replace(to_replace = 0.0, value = np.nan, inplace = True)

if det[D1].min() < 2 or det[D2].min() < 2:
    print "Rango de diametro tiene valores inferiores a 2 cm."
    print det[[D1, D2, PLOT]][(det[D1] < 2) | (det[D2] < 2)].join(info[['PLOT', 'SOCIO']
            ].set_index(PLOT), on=PLOT, rsuffix='_info')

#
# ‚¿Como tratar los valores de inclinación? ¿Están simplemente desfazados 90 grados?
#   
if det[INCL].min() < -90:
    print "Rango inclinacion tiene valores menores al valor permitido (-90)."
if det[INCL].max() > 90:
    print "Rango inclinacion sobrepasa el valor permitido (90)"
    
if det[PI_cm].min() < 0:
    print "Hay valores negativos de entrada del penetrometro."
if det[PI_cm].max() > 20:
    print "Valores maximos del entrada del penetrometro mayores al valor sugerido en el manual:"
    #print det[[PI_cm, D1, D2]][det[PI_cm] > 20]
    print det[[PI_cm, PLOT]][det[PI_cm] > 20].join(info[['PLOT', 'SOCIO']].set_index(PLOT), 
            on=PLOT, rsuffix='_info')

if len(det[PI_cm][(det[PI_cm] > det[D1]) | (det[PI_cm] > det[D1])]):
    print "Valores de entrada del penetrómetro son mayores al diametro registrado:"
    print det[[PI_cm, D1, D2, PLOT]][(det[PI_cm] > det[D1]) | (det[PI_cm] > det[D1])].join(
            info[['PLOT', 'SOCIO']].set_index(PLOT), on=PLOT, rsuffix='_info')

if det[PI_golpes].min() < 0:
    print "Valores negativos de golpes al penetrometro."
if det[PI_golpes].max() > 25:
    print "Valores maximos de golpes del penetrometro son dudosos:"
    #print det[PI_golpes][det[PI_golpes] > 20]
    print det[[PI_golpes, PLOT]][det[PI_golpes] > 20].join(info[['PLOT', 'SOCIO']].set_index(PLOT), 
            on=PLOT, rsuffix='_info')
    
# Sets de valores de espesor de pieza son dudosos si la desviacion estandar es mayor a 0.3 
# la media del set
if len(det[(det[[ESP1, ESP2, ESP3, ESP4]].std(1) / det[[ESP1, ESP2, ESP3, ESP4]].mean(1)) > 0.3]):
    print "Algunos conjuntos de espesor de pieza tienen una variacion muy alta:"
    print det[[ESP1, ESP2, ESP3, ESP4, PLOT]][(det[[ESP1, ESP2, ESP3, ESP4]].std(1) / det[[ESP1, 
                ESP2, ESP3, ESP4]].mean(1)) > 0.3].join(info[['PLOT', 'SOCIO']].set_index(PLOT), 
                on=PLOT, rsuffix='_info')
    
if det[(det[PESO_MUESTRA] < det[PESO_SECO]) | (det[PESO_MUESTRA].isna() & (det[PESO_RODAJA] < 
        det[PESO_SECO]))].size:
    print "Peso fresco es menor al peso seco:"
    print det[[PESO_MUESTRA, PESO_RODAJA, PESO_SECO, PLOT]][(det[PESO_MUESTRA] < det[PESO_SECO]) 
            | (det[PESO_MUESTRA].isna() & (det[PESO_RODAJA] < det[PESO_SECO]))].join(info[['PLOT', 
            'SOCIO']].set_index(PLOT), on=PLOT, rsuffix='_info')

if det[(det[PESO_MUESTRA] > det[PESO_RODAJA])].size:
    print "Peso del fragmento muestreado es mayor al de la rodaja:"
    print det[[TIPO, PESO_MUESTRA, PESO_RODAJA, PESO_SECO, PLOT]][(det[PESO_MUESTRA] > 
            det[PESO_RODAJA])].join(info[['PLOT', 'SOCIO']].set_index(PLOT), on=PLOT, 
            rsuffix='_info')
    
if det[det[PESO_MUESTRA].isna() & det[PESO_RODAJA].isna()].size:
    print "Falta peso fresco de detrito:"
    print det[[TIPO, PESO_MUESTRA, PESO_RODAJA, PESO_SECO, PLOT]][det[PESO_MUESTRA].isna() & 
            det[PESO_RODAJA].isna()].join(info[['PLOT', 'SOCIO']].set_index(PLOT), on=PLOT, 
            rsuffix='_info')

# Vegetacion

In [ ]:
CONS = "CONS" # Indice de medicion comun con arboles muertos en pie (int)
PLOT = "PLOT" # Indice conglomerado (int)
SPF = "SPF" # Indice subparcela (int 1-5)
IND = "IND" # Indice de individuo en el conglomerado (int)
TAMANO = "TAMANO" # Tamaño del individuo (str: 'L', 'F', o 'FG')
AZIMUT = "AZIMUT" # Orientacion del individuo desde el centro de la subparcela (int, 0-360)
DIST = "DIST" # Distancia en m del individuo al centro de la parcela (float, 0-15.74)
DAP1 = "DAP1" # Primer diámetro estimado del tallo en cm (float)
DAP2 = "DAP2" # Segundo diámetro estimado del tallo en cm (float)
DAPA = "DAPA" # Diametro promedio del tallo en cm (float)
ALTF = "ALTF" # Altura fuste en m (float)
ALTT = "ALTT" # Altura total en m (float)
FAMILIA = "FAMILIA" # Familia taxonomica (str)
GENERO = "GENERO" # Genero taxonomico (str)
EPITETO = "EPITETO" # Epiteto taxonomico (str)
AUTOR = "AUTOR" # Autor taxonomico (str)
ESPECIE = "ESPECIE" # Binomio taxonomico (str)
DENS = "DENS" # Densidad de la madera en gr/ml (float)
FUENTE_DENSIDAD = "FUENTE_DENSIDAD" # Referencia bibliografica de la densidad de la madera (str)

In [ ]:
for fi in [CONS, PLOT, SPF, IND]:
    if veg[fi].dtype != np.int64:
        print "Campo {0} tiene tipo inapropiado ({1} en vez de int64).".format(fi, veg[fi].dtype)
        if len(veg[fi][det[fi].isna()]) > 1:
            print "Valores nulos son considerados np.float64:"
            print veg[[fi, PLOT]][veg[fi].isna()].join(info[['PLOT', 'SOCIO']].set_index(PLOT), 
                                                       on=PLOT, rsuffix='_info')
        else:
            print "Valores np.float64 a revisar:"
            print veg[fi][veg[fi].map(lambda x: x % 1.0 != 0)].dropna()
            
        
for fi in [AZIMUT, DIST, DAP1, DAP2, DAPA, ALTT, ALTF, DENS]:
    if veg[fi].dtype != np.float64:
        print "Campo {0} tiene tipo inapropiado ({1}en vez de float64).".format(fi, veg[fi].dtype)

for fi in [FAMILIA, GENERO, EPITETO, AUTOR, ESPECIE, FUENTE_DENSIDAD, TAMANO]:
    non_strings = veg[fi].dropna()[~veg[fi].dropna().apply(type).eq(str)]
    if len(non_strings):
        print "Campo {0} tiene tipo inapropiado ({1} en vez de str).".format(fi, non_strings.dtype)

In [ ]:
if len(veg[veg[CONS].duplicated()]):
    print "Tabla {0} contiene indices duplicados.".format(vegetacion)

for spf in veg[SPF].unique():
    if spf not in range(1,6):
        print "Valor no valido de parcela: {0}".format(spf)
        
for tam in veg[TAMANO].unique():
    if tam not in ['L', 'F', 'FG']:
        print "Valor no valido de tamaño de individuo: {0}".format(tam)
        
if veg[AZIMUT].min() < 0:
    print "Azimut contiene valores no aceptados."
if veg[AZIMUT].max() > 360:
    print "Azimut contiene valores no aceptados."
    
# Verificar asignacion de clases diametricas
if len(veg[(veg[TAMANO] != 'B') & ((veg[DAPA] < 2.5) | ((veg[DAP1] + veg[DAP2]) < 5.0))]):
    print "Brinzales están asignados a categoria erronea:"
    print veg[[PLOT, TAMANO, DAPA, DAP1, DAP2]][(veg[TAMANO] != 'B') & ((veg[DAPA] < 2.5) 
            | ((veg[DAP1] + veg[DAP2]) < 5.0))].join(info[['PLOT', 'SOCIO']].set_index(PLOT),
            on=PLOT, rsuffix='_info')

if len(veg[(veg[TAMANO] != 'L') & (((veg[DAPA] < 10) & (veg[DAPA] >= 2.5)) | (((veg[DAP1] + 
        veg[DAP2]) < 20) & ((veg[DAP1] + veg[DAP2]) >= 5.0)))]):
    print "Latizales están asignados a categoria erronea:"
    print veg[[PLOT, TAMANO, DAPA, DAP1, DAP2]][(veg[TAMANO] != 'L') & (((veg[DAPA] < 10) & 
            (veg[DAPA] >= 2.5)) | (((veg[DAP1] + veg[DAP2]) < 20) & ((veg[DAP1] + 
            veg[DAP2]) >= 5.0)))].join(info[['PLOT', 'SOCIO']].set_index(PLOT), on=PLOT, 
            rsuffix='_info')
    
if len(veg[(veg[TAMANO] != 'F') & (((veg[DAPA] < 30) & (veg[DAPA] >= 10)) | (((veg[DAP1] + 
        veg[DAP2]) < 60) & ((veg[DAP1] + veg[DAP2]) >= 20)))]):
    print "Fustales están asignados a categoria erronea:"
    print veg[[PLOT, TAMANO, DAPA, DAP1, DAP2]][(veg[TAMANO] != 'F') & (((veg[DAPA] < 30) & 
            (veg[DAPA] >= 10)) | (((veg[DAP1] + veg[DAP2]) < 60) & ((veg[DAP1] + 
            veg[DAP2]) >= 20)))].join(info[['PLOT', 'SOCIO']].set_index(PLOT), on=PLOT, 
            rsuffix='_info')
    
if len(veg[(veg[TAMANO] != 'FG') & ((veg[DAPA] >= 30) | ((veg[DAP1] + veg[DAP2]) >= 60))]):
    print "Fustales grandes están asignados a categoria erronea:"
    print veg[[PLOT, TAMANO, DAPA, DAP1, DAP2]][(veg[TAMANO] != 'FG') & ((veg[DAPA] >= 30) 
            | ((veg[DAP1] + veg[DAP2]) >= 60))].join(info[['PLOT', 'SOCIO']].set_index(PLOT),
            on=PLOT, rsuffix='_info')

# Verificar si las distancias corresponden a las categorias de edad.
if len(veg[(veg[DIST] > 3) & (veg[TAMANO] == 'L')]):
    print "Latizales registrados afuera del area aceptada:"
    print veg[[TAMANO, DIST, DAPA, PLOT]][(veg[DIST] > 3) & (veg[TAMANO] == 'L')].join(info[[
            'PLOT', 'SOCIO']].set_index(PLOT), on=PLOT, rsuffix='_info')
    
if len(veg[(veg[DIST] > 7) & (veg[TAMANO] == 'F')]):
    print "Fustales registrados afuera del area aceptada:"
    print veg[[TAMANO, DIST, DAPA, PLOT]][(veg[DIST] > 7) & (veg[TAMANO] == 'F')].join(info[[
            'PLOT', 'SOCIO']].set_index(PLOT), on=PLOT, rsuffix='_info')
    
if len(veg[veg[DIST] > 15]):
    print "Individuos registrados afuera del area de la subparcela:"
    print veg[[TAMANO, DIST, DAPA, PLOT]][veg[DIST] > 15].join(info[['PLOT', 'SOCIO']]
            .set_index(PLOT), on=PLOT, rsuffix='_info')

# Verificar estimacion DAP promedio
if veg[((veg[DAP1] + veg[DAP2]) - (veg[DAPA] * 2)) > 0.1].size:
    print "Errores en la estimación del DAP promedio?:"
    print veg[[DAPA, DAP1, DAP2, PLOT]][((veg[DAP1] + veg[DAP2]) - (veg[DAPA] * 2)) > 
            0.01].join(info[['PLOT', 'SOCIO']].set_index(PLOT), on=PLOT, rsuffix='_info')

# Altura total siempre debe ser mayor a la altura del fuste
if veg[veg[ALTF] > veg[ALTT]].size:
    print "Individuos con altura del fuste mayor a la altura total:"
    print veg[[ALTF, ALTT, PLOT]][veg[ALTF] > veg[ALTT]].join(info[['PLOT', 'SOCIO']]
            .set_index(PLOT), on=PLOT, rsuffix='_info')

# Informacion general

In [11]:
CONS = u'CONS' # Indice informacion (int64)
PLOT = u'PLOT' # Indice conglomerado (int64)
DEPARTAMENTO = u'DEPARTAMENTO' # Departamento (str)
REGION = u'REGION' # Region biogeografica (str: 'Amazonia', 'Andes', 'Pacifico', 'Orinoquia', 'Caribe')
FECHA_CAMPO = u'FECHA_CAMPO' # Año de toma de datos (int64)
SOCIO = u'SOCIO' # Institucion que ejecuta el levantamiento de datos (str: 'Sinchi', 'IAvH', 'IIAP')
BOT_TOT = u'BOT_TOT' # ???????????????? (str: 'Si', 'No'). Deberia ser boolean.
CARB = u'CARB' # Estimacion de carbono ????? (str: 'Si', 'No'). Deberia ser boolean.
SPF = u'SPF-C' # ??????? (int64 0-5)
FERT = u'FERT' # Estimacion fertilidad ????????????? (str: 'Si', 'No'). Deberia ser boolean.
DETR = u'DETR' # Toma de detritos ????????? (str: 'Si', 'No'). Deberia ser boolean.

In [4]:
info.columns

Index([u'CONS', u'PLOT', u'DEPARTAMENTO', u'REGION', u'FECHA_CAMPO', u'SOCIO',
       u'BOT_TOT', u'CARB', u'SPF-C', u'FERT', u'DETR'],
      dtype='object')

In [20]:
# Verificar el tipo de dato an cada campo de la tabla informaciongeneral
for fi in [CONS, PLOT, FECHA_CAMPO, SPF]:
    if info[fi].dtype != np.int64:
        print "Campo {0} tiene tipo inapropiado ({1} en vez de int64).".format(fi, veg[fi].dtype)
        if len(info[fi][det[fi].isna()]) > 1:
            print "Valores nulos son considerados np.float64:"
            print info[[fi, PLOT, SOCIO]]
        else:
            print "Valores np.float64 a revisar:"
            print info[fi][info[fi].map(lambda x: x % 1.0 != 0)].dropna()
            
for fi in [DEPARTAMENTO, REGION, SOCIO]:
    non_strings = info[fi].dropna()[~info[fi].dropna().apply(type).eq(unicode)]
    if len(non_strings):
        print "Campo {0} tiene tipo inapropiado ({1} en vez de unicode).".format(fi, non_strings.dtype)

###########################################
# Se asume que los campos BOT_TOT, fertilidad y detritos son boolean en ves de texto
###########################################
try:
    info[BOT_TOT].replace(to_replace = [u'Si', u'No'], value = [True, False], inplace = True)
    info[FERT].replace(to_replace = [u'Si', u'No'], value = [True, False], inplace = True)
    info[DETR].replace(to_replace = [u'Si', u'No'], value = [True, False], inplace = True)
except TypeError, ErrorMessage:
    if ErrorMessage.args[0] == "Cannot compare types 'ndarray(dtype=bool)' and 'unicode'":
        pass
except:
    raise
    
for fi in [BOT_TOT, DETR, FERT]:
    if info[fi].dtype != np.bool:
        print "Campo {0} tiene tipo inapropiado ({1} en vez de np.bool).".format(fi, veg[fi].dtype)

In [ ]:
# Verificar rango de datos


In [19]:
info.FERT.unique()

array([ True, False], dtype=bool)